In [57]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.optimize import minimize
import api

# Section 1 -- Define tickers and time period
## How can i create a portfolio with the highest risk adjusted return?


## Define the list of tickers

In [3]:
tickers = ['SPY', 'BND','GLD','QQQ','VTI']

In [4]:
end_date = datetime.today()

## Set start date 5 years ago

In [5]:
start_date = end_date - timedelta(days=365*5)
start_date

# Section 2: download Adjusted Close Prices

## Create an empty DataFrame to store the adj close prices

### Adj close is more representative of overall returns as they include dividents and stock splits

In [6]:
adj_close_df = pd.DataFrame()

## Download close price for each of the tickers

In [26]:
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
    adj_close_df[ticker] = data['Adj Close']

In [27]:
# Check adj close_df
print(adj_close_df.head())

# Section 3: Calculate Log normal returns for each ticker

In [31]:
log_returns_df = np.log(adj_close_df / adj_close_df.shift(1))
# Check log returns
print(log_returns_df.head())
print(log_returns_df.count())

## Drop missing values

In [32]:
log_returns_df = log_returns_df.dropna()
log_returns_df.count()

# Section 4: Calculate Covariance Matrix

In [33]:
cov_matrix = log_returns_df.cov() * 252  # Annualize the covariance matrix
cov_matrix

# Section 5: Portfolio performance metrics


## Calculate Portfolio Standard deviation

In [34]:
def standard_deviation(weights, cov_matrix):
    """
    Calculate the portfolio standard deviation.
    """
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

## Calculate Expected Return

Assuming expected returns are similar to historical returns

In [35]:
def expected_return(weights, log_returns_df):
    """
    Calculate the expected portfolio return.
    """
    return np.sum(log_returns_df.mean() * weights) * 252  # Annualize the return

## Calculate Sharpe Ratio

In [36]:
def sharpe_ratio(weights, log_returns_df, cov_matrix, risk_free_rate):
    """
    Calculate the Sharpe Ratio of the portfolio.
    """
    return(expected_return (weights, log_returns_df) - risk_free_rate) / standard_deviation(weights, cov_matrix)

## Calculate Sortino Ratio

In [37]:
def sortino_ratio(weights, log_returns_df, cov_matrix, risk_free_rate):
    """
    Calculate the Sortino Ratio of the portfolio.
    """
    downside_returns = log_returns_df[log_returns_df < 0]
    downside_deviation = np.sqrt(np.dot(weights.T, np.dot(downside_returns.cov() * 252, weights)))
    return (expected_return(weights, log_returns_df) - risk_free_rate) / downside_deviation

# Section 6: Portfolio Optimization

## Set Risk Free Rate

In [58]:

def get_risk_free_rate(api_key):
    """
    Fetch the risk-free rate from FRED.
    """
    import requests
    #using the 10-year Treasury yield as a proxy for the risk-free rate
    url = f'https://api.stlouisfed.org/fred/series/observations?series_id=GS10&api_key={api.FRED_API_KEY}&file_type=json'
    response = requests.get(url)
    data = response.json()
    rate = float(data['observations'][-1]['value']) / 100  # Convert percentage to decimal
    return rate
risk_free_rate = get_risk_free_rate(fred_api_key)

In [59]:
risk_free_rate

## Define Function to minimize

### Negative sharpe ratio

we use the scipy.optimize.minimize() function to minimize the value

In [47]:
def negative_sharpe_ratio(weights, log_returns_df, cov_matrix, risk_free_rate):
    """
    Objective function to minimize: negative Sharpe Ratio.
    """
    return -sharpe_ratio(weights, log_returns_df, cov_matrix, risk_free_rate)

## Set Constrains and Bounds

constrains and conditions that must be met by the solution during the optimization process.

In [49]:
constrains = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})  # Weights must sum to 1

bounds = [(0, .5) for _ in range(len(tickers))]  # Weights must be between 0 and 1

## Set Initial weights

In [50]:
# can be changed to inputs by user

initial_weights = np.array([1/len(tickers)] * len(tickers))  # Equal weights

## Oprimize the weights to maximize the the sharpe ratio

In [51]:
optimized_result = minimize(negative_sharpe_ratio, initial_weights, args = (log_returns_df, cov_matrix, risk_free_rate),
                            method='SLSQP', bounds=bounds, constraints=constrains)

## Get optimal weights

In [52]:
optimal_weights = optimized_result.x


# Section 7: Analyze the Optimal Portfolio

## Display analytics of the optimal portfolio

In [53]:
print(optimal_weights)
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.2%}")
print()

optimal_portfolio_return = expected_return(optimal_weights, log_returns_df)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_portfolio_sharpe = sharpe_ratio(optimal_weights, log_returns_df, cov_matrix, risk_free_rate)

print(f"expected annual return: {optimal_portfolio_return: 4f}")
print(f"expected volatility: {optimal_portfolio_volatility: 4f}")
print(f"expected Sharpe Ratio: {optimal_portfolio_sharpe: 4f}")

## Display Final Portfolio as a Graph

In [56]:
# display as a pie chart using matplotlib
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.pie(optimal_weights, labels=tickers, autopct='%1.1f%%', startangle=140)
plt.title('Optimal Portfolio Allocation')
plt.axis('equal')  # Equal aspect ratio ensures that pie chart is a circle.
plt.show()